<h1>Task Manager</h1>

In [1]:
import sys
import os
import pandas as pd
from PyQt5.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QHBoxLayout, QPushButton,
    QListWidget, QInputDialog, QMessageBox
)

# Global task list
to_do = []

class ToDoApp(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Task Manager by Emilia Woldan")
        self.setGeometry(100, 100, 700, 400)
        self.layout = QVBoxLayout()

        self.task_list = QListWidget()
        self.layout.addWidget(self.task_list)

        # Buttons
        btn_layout = QHBoxLayout()
        self.layout.addLayout(btn_layout)

        buttons = [
            ("Add Task", self.add_task),
            ("Edit Task", self.edit_task),
            ("Mark Complete", self.mark_complete),
            ("Delete All", self.delete_all),
            ("Save", self.save_tasks),
            ("Load", self.load_tasks),
            ("Search", self.search_task),
            ("High Priority", self.show_high_priority),
            ("Sort A-Z", self.sort_tasks),
            ("Exit", self.close)
        ]

        for label, func in buttons:
            btn = QPushButton(label)
            btn.clicked.connect(func)
            btn_layout.addWidget(btn)

        self.setLayout(self.layout)
        self.refresh_list()

    def refresh_list(self):
        self.task_list.clear()
        for task in to_do:
            self.task_list.addItem(f"ID {task['id']} | {task['title']} | {task['status']} | {task['priority']}")

    def add_task(self):
        title, ok = QInputDialog.getText(self, "Add Task", "Enter task title:")
        if not ok or not title:
            return

        if any(task['title'].lower() == title.lower() for task in to_do):
            QMessageBox.warning(self, "Duplicate", "Task already exists.")
            return

        priority, ok = QInputDialog.getText(self, "Priority", "Enter priority (Low, Medium, High):")
        if not ok or not priority:
            return

        task_id = len(to_do) + 1
        to_do.append({
            "id": task_id,
            "title": title,
            "status": "Not started",
            "priority": priority.capitalize()
        })
        self.refresh_list()

    def edit_task(self):
        row = self.task_list.currentRow()
        if row == -1:
            QMessageBox.information(self, "Info", "Select a task first.")
            return

        task = to_do[row]

        new_title, ok = QInputDialog.getText(self, "Edit Title", "Enter new title:", text=task['title'])
        if ok and new_title:
            task['title'] = new_title

        new_status, ok = QInputDialog.getText(self, "Edit Status", "New status (Not started, In Progress, Completed):", text=task['status'])
        if ok and new_status:
            task['status'] = new_status.capitalize()

        new_priority, ok = QInputDialog.getText(self, "Edit Priority", "New priority (Low, Medium, High):", text=task['priority'])
        if ok and new_priority:
            task['priority'] = new_priority.capitalize()

        self.refresh_list()

    def mark_complete(self):
        row = self.task_list.currentRow()
        if row == -1:
            QMessageBox.information(self, "Info", "Select a task first.")
            return
        to_do[row]['status'] = "Completed"
        self.refresh_list()

    def delete_all(self):
        confirm = QMessageBox.question(self, "Confirm", "Are you sure you want to delete all tasks?")
        if confirm == QMessageBox.Yes:
            to_do.clear()
            self.refresh_list()

    def save_tasks(self):
        if not to_do:
            QMessageBox.information(self, "Info", "No tasks to save.")
            return
        df = pd.DataFrame(to_do)
        df.to_excel("tasks.xlsx", index=False)
        QMessageBox.information(self, "Saved", "Tasks saved to 'tasks.xlsx'.")

    def load_tasks(self):
        global to_do
        if not os.path.exists("tasks.xlsx"):
            QMessageBox.warning(self, "Not Found", "No saved file found.")
            return
        df = pd.read_excel("tasks.xlsx")
        to_do = df.to_dict(orient="records")
        self.refresh_list()
        QMessageBox.information(self, "Loaded", "Tasks loaded successfully.")

    def search_task(self):
        keyword, ok = QInputDialog.getText(self, "Search", "Enter keyword:")
        if not ok or not keyword:
            return
        results = [task for task in to_do if keyword.lower() in task['title'].lower()]
        self.task_list.clear()
        if not results:
            QMessageBox.information(self, "Search", "No matching tasks found.")
            return
        for task in results:
            self.task_list.addItem(f"ID {task['id']} | {task['title']} | {task['status']} | {task['priority']}")

    def show_high_priority(self):
        high = [task for task in to_do if task['priority'].lower() == "high"]
        self.task_list.clear()
        if not high:
            QMessageBox.information(self, "High Priority", "No high priority tasks.")
            return
        for task in high:
            self.task_list.addItem(f"ID {task['id']} | {task['title']} | {task['status']} | {task['priority']}")

    def sort_tasks(self):
        to_do.sort(key=lambda t: t['title'].lower())
        self.refresh_list()


if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = ToDoApp()
    window.show()
    sys.exit(app.exec_())


SystemExit: 0

/Users/emiliawoldan/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3377: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
